In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#define data
data = [[1, 11], 
        [2, 8], 
        [3, 4], 
        [4, 5], 
        [5, 5], 
        [6, 8],
        [7, 7],
        [8, 7],
        [9, 6],
        [10, 4]] 
  
#define column names
columns = ['day', 'sales']

#create dataframe using data and column names
df = spark.createDataFrame(data, columns) 
  
#view dataframe
df.show()



25/12/20 20:32:04 WARN Utils: Your hostname, user-HP-Pavilion-x360-Convertible-14-dh0xxx resolves to a loopback address: 127.0.1.1; using 192.168.1.24 instead (on interface wlo1)
25/12/20 20:32:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/20 20:32:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                

+---+-----+
|day|sales|
+---+-----+
|  1|   11|
|  2|    8|
|  3|    4|
|  4|    5|
|  5|    5|
|  6|    8|
|  7|    7|
|  8|    7|
|  9|    6|
| 10|    4|
+---+-----+



In [3]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

#define window for calculating rolling mean
# w = (Window.orderBy('day').rowsBetween(-3, 0)) 
w = (Window.orderBy('day').rowsBetween(Window.currentRow-3, Window.currentRow)) 
#create new DataFrame that contains 4-day rolling mean column
df_new = df.withColumn('rolling_mean', F.avg('sales').over(w))

#view new DataFrame
df_new.show()

25/12/20 20:33:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/20 20:33:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/20 20:33:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---+-----+-----------------+
|day|sales|     rolling_mean|
+---+-----+-----------------+
|  1|   11|             11.0|
|  2|    8|              9.5|
|  3|    4|7.666666666666667|
|  4|    5|              7.0|
|  5|    5|              5.5|
|  6|    8|              5.5|
|  7|    7|             6.25|
|  8|    7|             6.75|
|  9|    6|              7.0|
| 10|    4|              6.0|
+---+-----+-----------------+



25/12/20 20:33:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/20 20:33:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


from pyspark.sql.window import Window
Supported Methods in the Window Class
The Window class provides three primary methods to create a "Window Specification" (WindowSpec). These define the boundaries and logic for your window functions.

|Method |Description|
|--------|----------|
|partitionBy(*cols)|Groups the data into logical buckets (partitions). Functions are applied independently within each bucket.|
|orderBy(*cols)|Defines the sequence of rows within each partition. This is required for ranking and analytical functions.|
|"rowsBetween(start, end)"|"Defines a frame based on the position of rows relative to the current row (e.g., ""1 row before to current row"")."|
|"rangeBetween(start, end)"|"Defines a frame based on the actual values of the order-by column (e.g., ""all rows with a date within 7 days of the current row"")."|

When using rowsBetween or rangeBetween, you often use these constants to define infinite boundaries:

* Window.unboundedPreceding: The very first row in the partition.
* Window.unboundedFollowing: The very last row in the partition.
* Window.currentRow: The current row being processed.

In [ ]:
can you write down PySpark code "rangeBetween(start, end)"	"Defines a frame based on the actual values of the order-by column (e.g., ""all rows with a date within 7 days of the current row"")."

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("RangeBetweenExample").getOrCreate()

# Sample Data: Sales over several days
data = [
    ("Product_A", "2023-01-01", 100),
    ("Product_A", "2023-01-02", 150),
    ("Product_A", "2023-01-05", 200),  # 3 days after previous
    ("Product_A", "2023-01-10", 100),  # 5 days after previous
    ("Product_A", "2023-01-11", 50),
]

df = spark.createDataFrame(data, ["product", "date", "amount"])

# 1. Convert date string to Unix Timestamp (seconds)
df = df.withColumn("timestamp", F.unix_timestamp(F.col("date"), "yyyy-MM-dd"))
df.show()
# 2. Define the range in seconds (7 days = 7 * 24 * 60 * 60)
seconds_in_7_days = 7 * 24 * 60 * 60

# 3. Create the Window Specification
# rangeBetween(start, end) looks at the VALUE of the orderBy column
# -seconds_in_7_days: Look back 7 days worth of seconds
# Window.currentRow: Stop at the current timestamp
windowSpec = Window.partitionBy("product") \
                   .orderBy("timestamp") \
                   .rangeBetween(-seconds_in_7_days, Window.currentRow)

# 4. Apply the window
df_result = df.withColumn("7_day_rolling_sum", F.sum("amount").over(windowSpec))

df_result.select("product", "date", "amount", "7_day_rolling_sum").show()

In [ ]:
df.show()

In [ ]:
df.orderBy("date").show()